In [3]:
library(spacexr)
library(Matrix)
library(Seurat)

Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect




In [4]:
load("visium.RData")
load("scall.RData")
sc.all <- UpdateSeuratObject(scall)

In [14]:
sc.week6 <- subset(sc.all, subset = time == "6weeks")

In [10]:
sp <- subset(visium, subset = ident == "Week6R")

In [17]:
library(dplyr)

sc.week6@meta.data <- sc.week6@meta.data %>%
  mutate(name = case_when(
    name %in% c("MTAL", "CTAL1", "CTAL2") ~ "TAL",
    name %in% c("CNT", "DCT-CNT") ~ "CNT",
    name %in% c("EC1", "EC2") ~ "EC",
    name %in% c("PC1", "PC2") ~ "PC",
    name %in% c("NewPT1") ~ "Inj_PT",
    name %in% c("NewPT2") ~ "FR_PT",
    name %in% c("Mø", "Tcell") ~ "Immune",
    TRUE ~ name 
  ))


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [25]:
sc.celltype <- sc.week6@meta.data$name
names(sc.celltype) <- rownames(sc.week6@meta.data)
sc.celltype <- as.factor(sc.celltype)

In [27]:
sc.counts <- sc.week6@assays$RNA$counts

In [28]:
sc_reference=Reference(
    counts=sc.counts,
    cell_types=sc.celltype
)

In [35]:
sp.counts <- sp@assays$RNA$counts
sp.location <- sp@meta.data[c("x_align","y_align")]

In [37]:
st_data=SpatialRNA(
    counts=sp.counts,
    coords=sp.location,
    require_int=FALSE
)

In [38]:
myRCTD <- create.RCTD(st_data, sc_reference, max_cores = 1, CELL_MIN_INSTANCE = 1)

Begin: process_cell_type_info

process_cell_type_info: number of cells in reference: 23473

process_cell_type_info: number of genes in reference: 27133




    CNT     DCT DTL-ATL      EC     Fib   FR_PT     ICA     ICB  Immune  Inj_PT 
   1331    1204     874    1751     977     552     825     410     946     189 
     MD      PC     PEC     Per     Pod    PTS1    PTS2    PTS3     TAL     Uro 
     34    1580     104      95     154    2897    2812    1511    5001     226 


Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 1.0 GiB”
End: process_cell_type_info

create.RCTD: getting regression differentially expressed genes: 

get_de_genes: CNT found DE genes: 141

get_de_genes: DCT found DE genes: 125

get_de_genes: DTL-ATL found DE genes: 149

get_de_genes: EC found DE genes: 271

get_de_genes: Fib found DE genes: 243

get_de_genes: FR_PT found DE genes: 154

get_de_genes: ICA found DE genes: 177

get_de_genes: ICB found DE genes: 177

get_de_genes: Immune found DE genes: 315

get_de_genes: Inj_PT found DE genes: 96

get_de_genes: MD found DE genes: 222

get_de_genes: PC found DE genes: 173

get_de_genes: PEC found DE genes: 208

get_de_genes: Per found DE genes: 267

get_de_genes: Pod found DE genes: 221

get_de_genes: PTS1 found DE genes: 222

get_de_genes: PTS2 found DE genes: 257

get_de_genes: PTS3 found DE genes: 264

get_de_genes: TAL found DE genes: 118

get_de_genes: Uro found DE genes: 211

get_de_genes: tot

In [39]:
myRCTD <- run.RCTD(myRCTD, doublet_mode = 'doublet')

fitBulk: decomposing bulk

chooseSigma: using initial Q_mat with sigma =  1

Likelihood value: 2749047.29756562

Sigma value:  0.84

Likelihood value: 2689908.37295716

Sigma value:  0.69

Likelihood value: 2640965.80739773

Sigma value:  0.61

Likelihood value: 2618825.24671313

Sigma value:  0.53

Likelihood value: 2600455.92501149

Sigma value:  0.45

Likelihood value: 2586851.52548375

Sigma value:  0.37

Likelihood value: 2579220.46969544

Sigma value:  0.34

Likelihood value: 2578189.34644411

Sigma value:  0.33



[1] "gather_results: finished 1000"


In [40]:
weights=myRCTD@results$weights

In [42]:
norm_weights <- normalize_weights(weights)

In [43]:
norm_weights

1799 x 20 Matrix of class "dgeMatrix"
                     CNT          DCT      DTL-ATL         EC          Fib
Week6R_0    4.447404e-05 6.137321e-03 9.431310e-04 0.06153037 3.787186e-02
Week6R_1    4.378816e-05 3.766821e-03 4.378816e-05 0.10430827 1.044110e-01
Week6R_2    3.627416e-05 6.749763e-03 2.886237e-02 0.21954787 1.209931e-01
Week6R_3    2.870945e-05 1.121366e-02 2.870945e-05 0.07600792 3.759341e-02
Week6R_4    4.336759e-05 5.703900e-02 4.336759e-05 0.07486917 1.259747e-03
Week6R_5    5.483062e-05 1.700218e-02 7.556505e-02 0.06130166 4.909187e-02
Week6R_6    2.474680e-03 4.383252e-05 4.383252e-05 0.11063947 5.209698e-02
Week6R_7    3.070981e-05 1.927580e-02 3.070981e-05 0.07384850 2.110172e-02
Week6R_8    2.348913e-02 7.309794e-02 4.252917e-05 0.06255068 5.914344e-02
Week6R_9    3.239632e-01 4.402228e-02 5.793102e-05 0.11699529 5.793102e-05
Week6R_10   4.495044e-05 1.352937e-01 4.495044e-05 0.07945511 4.495044e-05
Week6R_11   3.678004e-05 2.152158e-02 2.619607e-01 0.14398435 

In [44]:
write.csv(as.matrix(norm_weights),"deconv_results/RCTD_deconv.csv")